### Goal

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

# to prevent figure 
# to show in the notebook
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

import plotting as plotting

### TODO: update this (GROUP OF 5 VERSION)

In [3]:
date = "24-12-16"
pathfile = f"Pulling data/{date}/groups_of_5/single/"

### From here; automatic run

In [4]:
root = f"/Users/michaelmoret/Library/CloudStorage/GoogleDrive-michael@externa.bio/.shortcut-targets-by-id/1BdUNsBjDh5Gee_76jCiKB1C_CwG0ercP/{pathfile}"

MODE = "single"
dev = False

# number of hairs from the same group together
n_together = 5
total_hairs = 100

In [5]:
if not root.endswith("/"):
    root += "/"

sep_category = "$" # separates categories like date, experiments, etc
sep_count = "%" # separated the number of hairs by experiment

In [6]:
all_dfs = []
all_fns = []

for file in os.listdir(root):
    if file.endswith(".txt"):
        print("\n*****************************")
        print(file)
        df = plotting.get_df_from_file(root + file,
                                       skip=10)
        # clean the df
        if MODE in file:
            df = plotting.clean_single(df)
            assert "single" in file
        else:
            raise ValueError(f"Not a {MODE} experiment")

        splitted_name = file.split(sep_category)
        # get the experiments; i.e. not the date
        # not the single or triple etc
        splitted_name = splitted_name[2:-1]
        print("splitted_name: ", splitted_name)
        # add the experiment name in the dataframe
        all_names = []
        
        # trick to get the right labels
        labelling_round = 0
        while labelling_round!=total_hairs:
            for entries in splitted_name:
                times_name = entries.split(sep_count)
                _name = times_name[1]
                all_names += [_name] * n_together
                labelling_round += n_together
                
        print(f"len df: {len(df)}, len names: {len(all_names)}")
        df["Name"] = all_names
        # remove nan
        df = df.dropna()
        all_dfs.append(df)
        all_fns.append(file.replace(".txt", ""))

print(all_names)


*****************************
20241216-repeat$IHNY$on25%phosctrl$on25% PK30-2 0.01$on25%PK30-2  0.03$on25% PK30-2 0.05$single.txt
Missing records: []
splitted_name:  ['on25%phosctrl', 'on25% PK30-2 0.01', 'on25%PK30-2  0.03', 'on25% PK30-2 0.05']
len df: 100, len names: 100

*****************************
20241216$IHNY$25%AActrl$25% PK30-2 0.01$25%PK30-2  0.03$25% PK30-2 0.05$single.txt
Missing records: [77]
Records order check  [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
splitted_name:  ['25%AActrl', '25% PK30-2 0.01', '25%PK30-2  0.03', '25% PK30-2 0.05']
len df: 100, len names: 100

*****************************
20241213$IHNY$25%phosctrl$25% PK30-2 0.01$25%PK30-2  0.03$25% PK30-2 0.05$single.txt
Missing records: []
splitted_name:  ['25%phosctrl', '25% PK30-2 0.01', '25%PK30-2  0.03', '25% PK30-2 0.05']
len df: 100, len names: 100
['phosctrl', 'phosctrl', 'phosctrl', 'phosctrl', 'phosctrl', ' PK30-2 0.01', ' PK30-2 0.01', ' PK30-2 0.01', ' PK30-2 0.01', ' PK30-2 0.01', 'PK30-2  0.03', 'PK30-2  0.

In [7]:
def master_plots(df, header, ymin, ymax, savepath, show=False, verbose=False):
    fig, ax, removed = plotting.create_boxplot(df, header, ymin, ymax)
    #save removed data
    removed.to_csv(f"{savepath}{header}_removed.csv", index=True)
    if verbose:
        print(f"{len(removed)} removed data points for {header}")
    # plot
    fig.savefig(f"{savepath}{header}.png")
    if show: 
        plt.show()
        plt.close()

In [8]:
len(all_dfs), len(all_fns)

(3, 3)

In [9]:
all_fns

['20241216-repeat$IHNY$on25%phosctrl$on25% PK30-2 0.01$on25%PK30-2  0.03$on25% PK30-2 0.05$single',
 '20241216$IHNY$25%AActrl$25% PK30-2 0.01$25%PK30-2  0.03$25% PK30-2 0.05$single',
 '20241213$IHNY$25%phosctrl$25% PK30-2 0.01$25%PK30-2  0.03$25% PK30-2 0.05$single']

In [10]:
for df, name in zip(all_dfs, all_fns):
    print("\n************************************")
    print(f"{name}\n")
    print(f"df length: {len(df)}")
    try:
        savepath = f"{root}{name}/"
        if dev:
            savepath = f"{root}dev/"
        os.makedirs(savepath, exist_ok=True)
        
        try:
            header = 'TENSILE_STRENGTH'
            ymin = 90
            ymax = 325
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'BREAK_STRAIN(*)(#)'
            ymin = 0
            ymax = 100
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'BREAK_LOAD'
            ymin = 0
            ymax = 2.0
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)
        
        try:
            header = 'EMOD(*)(#)'
            ymin = 2.5
            ymax = 8.0
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        # scatter plots
        y_col = 'TENSILE_STRENGTH'
        x_col = 'MEAN AREA'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'EMOD(*)(#)'
        x_col = 'MEAN AREA'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'BREAK_LOAD'
        x_col = 'MEAN AREA'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

    except:
        print(f"\nERROR with {name}\n")
        continue

print("\n\nplotting DONE")


************************************
20241216-repeat$IHNY$on25%phosctrl$on25% PK30-2 0.01$on25%PK30-2  0.03$on25% PK30-2 0.05$single

df length: 90

Creating boxplot for TENSILE_STRENGTH
Removed outliers for phosctrl : Series([], Name: TENSILE_STRENGTH, dtype: float64)
Removed outliers for  PK30-2 0.01 : Series([], Name: TENSILE_STRENGTH, dtype: float64)
Removed outliers for PK30-2  0.03 : 50    225.8
Name: TENSILE_STRENGTH, dtype: float64
Removed outliers for  PK30-2 0.05 : 37    229.5
56    146.7
Name: TENSILE_STRENGTH, dtype: float64
Stats. significance: PK30-2  0.03 and  PK30-2 0.05: p = 0.0253
Median for phosctrl: 194.65
Median for  PK30-2 0.01: 193.4
Median for PK30-2  0.03: 189.75
Median for  PK30-2 0.05: 200.45

Creating boxplot for BREAK_STRAIN(*)(#)
Removed outliers for phosctrl : Series([], Name: BREAK_STRAIN(*)(#), dtype: float64)
Removed outliers for  PK30-2 0.01 : 87    64.371
Name: BREAK_STRAIN(*)(#), dtype: float64
Removed outliers for PK30-2  0.03 : Series([], Name: B

/Users/michaelmoret/tinyverse/krap/notebooks_2/plotting.py:218: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=figsize)




plotting DONE
